# (prototype) Join test results 2

The first prototype joining test results resulted in missing rows. We want those rows to be present, but with their missing data as NaN. They mean that 5 or less students were in the category and so information was supressed.

In [1]:
import pandas as pd

pd.set_option('display.max_columns', None)

In [2]:
COLUMNS = [
    'DBN',
    'School Name',
    'Grade',
    'Year',
    'Category',
    'Number Tested',
    'Mean Scale Score',
    '# Level 1',
    '% Level 1',
    '# Level 2',
    '% Level 2',
    '# Level 3',
    '% Level 3',
    '# Level 4',
    '% Level 4',
    '# Level 3+4',
    '% Level 3+4'
]

def read_ela(in_ela_excel):
    df = pd.read_excel(in_ela_excel, sheet_name='All Students', skiprows=7, na_values='s')
    df = df.iloc[:,1:]  # drop first column
    df.columns = COLUMNS
    return df

def read_math(in_math_excel):
    df = pd.read_excel(in_math_excel, sheet_name='All Students', skiprows=7, na_values='s')
    df = df.iloc[:,1:]  # drop first column
    df.columns = COLUMNS
    return df

def read_charters(in_charter_excel):
    charters = pd.read_excel(in_charter_excel, sheet_name=['ELA', 'Math'], skiprows=7, na_values='s')
    charters['ELA'].columns = COLUMNS
    charters['Math'].columns = COLUMNS
    return charters


ela = read_ela('../data/raw/ela_results_20132017.xlsx')
math = read_math('../data/raw/math_results_20132017.xlsx')

charters = read_charters('../data/raw/charter_results_20132017.xlsx')
c_ela = charters['ELA']
c_math = charters['Math']

In [3]:
# remove unused columns
ela = ela.drop(['School Name', 'Category'], axis=1)
math = math.drop(['School Name', 'Category'], axis=1)
c_ela = c_ela.drop(['School Name', 'Category'], axis=1)
c_math = c_math.drop(['School Name', 'Category'], axis=1)

In [4]:
# create charter school flag

ncharter_dbn = set(ela['DBN']) | set(math['DBN'])
charter_flags1 = pd.Series(0, index=ncharter_dbn)

charter_dbn = set(c_ela['DBN']) | set(c_math['DBN'])
charter_flags2 = pd.Series(1, index=charter_dbn)

charter_flags = pd.concat([charter_flags1, charter_flags2])
charter_flags.name = 'Charter School?'
charter_flags.index.name = 'DBN'
charter_flags.head()

DBN
16K262    0
12X134    0
12X047    0
11X355    0
12X196    0
Name: Charter School?, dtype: int64

In [5]:
# create empty DataFrame with correct index

DBN = sorted(charter_flags.index)
Grade = [3, 4, 5, 6, 7, 8, 'All Grades']
Year = [2013, 2014, 2015, 2016, 2017]

index = pd.MultiIndex.from_product([DBN, Grade, Year], names=['DBN', 'Grade', 'Year'])
base_df = pd.DataFrame(index=index)
base_df.head()

Empty DataFrame
Columns: []
Index: [(01M015, 3, 2013), (01M015, 3, 2014), (01M015, 3, 2015), (01M015, 3, 2016), (01M015, 3, 2017)]

In [6]:
base_df.join(charter_flags).tail()

Charter School?
DBN    Grade      Year                 
84X730 All Grades 2013                1
                  2014                1
                  2015                1
                  2016                1
                  2017                1

In [7]:
# concatenate grades
f_ela = pd.concat([ela, c_ela]).set_index(['DBN', 'Grade', 'Year'])
f_math = pd.concat([math, c_math]).set_index(['DBN', 'Grade', 'Year'])

# join everything
grades = f_ela.join(f_math, how='outer', lsuffix=' - ELA', rsuffix=' - Math')
everything = base_df.join(charter_flags).join(grades)
everything.head()

Charter School?  Number Tested - ELA  \
DBN    Grade Year                                         
01M015 3     2013                0                 27.0   
             2014                0                 18.0   
             2015                0                 16.0   
             2016                0                 20.0   
             2017                0                 27.0   

                   Mean Scale Score - ELA  # Level 1 - ELA  % Level 1 - ELA  \
DBN    Grade Year                                                             
01M015 3     2013                289.2963             14.0         51.85185   
             2014                285.1111             10.0         55.55556   
             2015                281.8125              9.0         56.25000   
             2016                292.5000             10.0         50.00000   
             2017                302.3704             10.0         37.03704   

                   # Level 2 - ELA  % Level 2 - ELA  # Level 3 - ELA  \
DBN    Grade Year                                                      
01M015 3     2013             11.0         40.74074              2.0   
             2014              8.0         44.44444              0.0   
             2015              5.0         31.25000              2.0   
             2016              6.0         30.00000              4.0   
             2017              8.0         29.62963              7.0   

                   % Level 3 - ELA  # Level 4 - ELA  % Level 4 - ELA  \
DBN    Grade Year                                                      
01M015 3     2013         7.407407              0.0         0.000000   
             2014         0.000000              0.0         0.000000   
             2015        12.500000              0.0         0.000000   
             2016        20.000000              0.0         0.000000   
             2017        25.925930              2.0         7.407407   

                   # Level 3+4 - ELA  % Level 3+4 - ELA  Number Tested - Math  \
DBN    Grade Year                                                               
01M015 3     2013                2.0           7.407407                  27.0   
             2014                0.0           0.000000                  18.0   
             2015                2.0          12.500000                  17.0   
             2016                4.0          20.000000                  21.0   
             2017                9.0          33.333330                  29.0   

                   Mean Scale Score - Math  # Level 1 - Math  \
DBN    Grade Year                                              
01M015 3     2013                 277.7778              16.0   
             2014                 286.3889               6.0   
             2015                 279.5882              10.0   
             2016                 274.8095              13.0   
             2017                 301.5517               8.0   

                   % Level 1 - Math  # Level 2 - Math  % Level 2 - Math  \
DBN    Grade Year                                                         
01M015 3     2013          59.25926              11.0          40.74074   
             2014          33.33333               9.0          50.00000   
             2015          58.82353               4.0          23.52941   
             2016          61.90476               4.0          19.04762   
             2017          27.58621               9.0          31.03448   

                   # Level 3 - Math  % Level 3 - Math  # Level 4 - Math  \
DBN    Grade Year                                                         
01M015 3     2013               0.0           0.00000               0.0   
             2014               2.0          11.11111               1.0   
             2015               2.0          11.76471               1.0   
             2016               4.0          19.04762               0.0   
             2017               7.0          24.

In [8]:
everything.isnull().mean()

Charter School?            0.000000
Number Tested - ELA        0.404406
Mean Scale Score - ELA     0.405507
# Level 1 - ELA            0.405507
% Level 1 - ELA            0.405507
# Level 2 - ELA            0.405507
% Level 2 - ELA            0.405507
# Level 3 - ELA            0.405507
% Level 3 - ELA            0.405507
# Level 4 - ELA            0.405507
% Level 4 - ELA            0.405507
# Level 3+4 - ELA          0.405507
% Level 3+4 - ELA          0.405507
Number Tested - Math       0.407040
Mean Scale Score - Math    0.408487
# Level 1 - Math           0.408487
% Level 1 - Math           0.408487
# Level 2 - Math           0.408487
% Level 2 - Math           0.408487
# Level 3 - Math           0.408487
% Level 3 - Math           0.408487
# Level 4 - Math           0.408487
% Level 4 - Math           0.408487
# Level 3+4 - Math         0.408487
% Level 3+4 - Math         0.408487
dtype: float64

Lots of missing entries, most of these are schools without a given grade or schools where few students take a test.

In [9]:
# fix values for missing rows

everything.loc[everything['Number Tested - ELA'].isnull(), 'Number Tested - ELA'] = 0
everything.loc[everything['Number Tested - Math'].isnull(), 'Number Tested - Math'] = 0

In [10]:
everything.loc['17K590']

Charter School?  Number Tested - ELA  Mean Scale Score - ELA  \
Grade      Year                                                                 
3          2013                0                  0.0                     NaN   
           2014                0                  0.0                     NaN   
           2015                0                  0.0                     NaN   
           2016                0                  0.0                     NaN   
           2017                0                  0.0                     NaN   
4          2013                0                  0.0                     NaN   
           2014                0                  0.0                     NaN   
           2015                0                  0.0                     NaN   
           2016                0                  0.0                     NaN   
           2017                0                  0.0                     NaN   
5          2013                0                  0.0                     NaN   
           2014                0                  0.0                     NaN   
           2015                0                  0.0                     NaN   
           2016                0                  0.0                     NaN   
           2017                0                  0.0                     NaN   
6          2013                0                 99.0                321.7172   
           2014                0                101.0                330.2277   
           2015                0                104.0                334.1346   
           2016                0                 94.0                335.2021   
           2017                0                118.0                329.9661   
7          2013                0                 82.0                325.6951   
           2014                0                 92.0                322.0543   
           2015                0                 96.0                326.8438   
           2016                0                 99.0                333.7273   
           2017                0                 90.0                342.7000   
8          2013                0                 68.0                323.4265   
           2014                0                 79.0                328.3291   
           2015                0                 90.0                327.2444   
           2016                0                 92.0                334.7174   
           2017                0                 94.0                343.1383   
All Grades 2013                0                249.0                323.4940   
           2014                0                272.0                326.9118   
           2015                0                290.0                329.5828   
           2016                0                285.0                334.5333   
           2017                0                302.0                337.8609   

                 # Level 1 - ELA  % Level 1 - ELA  # Level 2 - ELA  \
Grade      Year                                                      
3          2013              NaN              NaN              NaN   
           2014              NaN              NaN              NaN   
           2015              NaN              NaN              NaN   
           2016              NaN              NaN              NaN   
           2017              NaN              NaN              NaN   
4          2013              NaN              NaN              NaN   
           2014              NaN              NaN              NaN   
           2015              NaN              NaN              NaN   
           2016              NaN              NaN              NaN   
           2017              NaN              NaN              NaN   
5          2013              NaN              NaN              NaN   
           2014              NaN              NaN              NaN   
           2015              NaN       

Finished